In [247]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import re
import requests
from datetime import datetime, timedelta

In [333]:
chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac-119/chromedriver'
chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)

driver = webdriver.Chrome(service=chrome_driver_service)
driver.maximize_window()
driver.get("https://www.ticketmelon.com")

wait = WebDriverWait(driver, 10) #timeout=10 sec

scroll_distance = 1200
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

print("scroll to entertainment :" , scroll_distance, "px")

entertainment_button = wait.until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.slick-slide.slick-active.slick-current')))
entertainment_button.click()

scroll_distance = 1500
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

print("scroll to see more :" , scroll_distance, "px")

see_more_button = driver.find_element(
    By.CSS_SELECTOR, '.HomeStyled__EventSeeMore-sc-125xn4a-2.eMhTTB')
driver.execute_script("arguments[0].click();", see_more_button)

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

driver.quit()

scroll to entertainment : 1200 px
scroll to see more : 1500 px


# initial df creation

In [ ]:
print(soup.prettify())

In [334]:
elements = soup.find_all('div',
                         class_='MelonEventCardStyled__EventCardContainer-sc-8kc3ju-0'
                         )  #event-date, event-name, event-location
event_data = []
for element in elements:

    event_info = {
        "title": element.find('p', class_='event-name').text,
        "date": element.find('p', class_='event-date').text,
        "location": element.find('p', class_='event-location').text,
        "url": "https://www.ticketmelon.com" + element.find('a').get('href')
    }
    event_data.append(event_info)

In [335]:
df = pd.DataFrame(event_data)
# display(df)

In [336]:
df = pd.DataFrame(event_data)

def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip() 
df[['startdate', 'enddate']] = df['date'].apply(extract_dates).apply(pd.Series)

def calculate_year(date_str):
    # Convert month abbreviations to numeric values
    month_values = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    # Extract the month from the date
    month = date_str.split()[1]
    
    if month_values.get(month):
        if month_values[month] >= 8:
            return 2023
        else:
            return 2024
    else:
        return None  # Handle invalid month

df['startyear'] = df['startdate'].apply(calculate_year)
df['endyear'] = df['enddate'].apply(calculate_year)
df['startyear'] = df['startyear'].astype(str)
df['endyear'] = df['endyear'].astype(str)

df['startdate'] = df['startdate'] + ' ' + df['startyear']
df['startdate'] = pd.to_datetime(df['startdate'], format='%d %b %Y', errors='coerce')
df['enddate'] = df['enddate'] + ' ' + df['endyear']
df['enddate'] = pd.to_datetime(df['enddate'], format='%d %b %Y', errors='coerce')

df.drop(columns=['startyear', 'endyear'], inplace=True)
df = df.sort_values(by=['startdate', 'enddate'])

df.drop(columns=['date'], inplace=True)

In [337]:
df.drop_duplicates()

,title,location,url,startdate,enddate
5,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07
6,Enchanted Flora Park,Flora Park,https://www.ticketmelon.com/florapark/enchante...,2023-11-01,2024-02-29
7,The Backyard 168: Back to the 90's Campfire Party,"The Backyard 168, Khao Yai",https://www.ticketmelon.com/thebackyard168/the...,2023-12-01,2024-02-25
8,GOODMOOD,Sermsuk Warehouse,https://www.ticketmelon.com/goodhood/goodmood,2023-12-02,2023-12-17
9,GOODHOOD VOLUME 04,Sermsuk Warehouse,https://www.ticketmelon.com/goodhood/goodhoodv...,2023-12-08,2023-12-17
...,...,...,...,...,...
126,SUPALAPA Festival,"Resorts World Awana, Genting Highlands, Malaysia",https://www.ticketmelon.com/supalapa/supalapaf...,2024-03-02,2024-03-03
127,Citizen Live In Bangkok 2024,Mr.fox,https://www.ticketmelon.com/loudlyprefer/citiz...,2024-03-10,2024-03-10
128,SLASH featuring Myles Kennedy Live in Bangkok,CENTERPOINT STUDIO,https://www.ticketmelon.com/pmg/slash15march2024,2024-03-15,2024-03-15
129,Southside Festival,Andamanda Phuket,https://www.ticketmelon.com/southsidepk/souths...,2024-03-30,2024-03-30


# add zone, price, solddate

In [326]:
def getSoup(url) :
    chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac-119/chromedriver'
    chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)
    driver = webdriver.Chrome(service=chrome_driver_service)
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    time.sleep(5)
    driver.quit()
    return soup

def getInfo(soup):
    types = []
    prices = []
    qtys = []

    elements = soup.find_all('div', class_='ticket--type')
    for element in elements :
        div_text = element.contents[0].strip()
        types.append(div_text)

    elements = soup.find_all('div', class_='ticket--price')
    for element in elements :
        prices.append(element.text)

    elements = soup.find_all('div', class_='ticket--qty')
    
    for element in elements :
        qtys.append(element.text)
    return types, prices, qtys

def createTypelist(title, types, prices, qtys):
    type_list = []
    for i in range(len(types)) :
        if prices[0].strip(" ") != 'Price' : continue #skip event that's not concert
        if (types[i] == 'Ticket Type') or (prices[i].strip(" ") =='Price') : continue #prevent header 

        #if SOLDOUT --> get today, if sale ended --> sale ended, else none
        solddate = None
        if qtys[i] == 'Sales Ended' : solddate = 'Sales Ended'
        elif qtys[i] == 'Sold Out' : solddate = datetime.now().strftime("%Y-%m-%d")

        type_info = {
            'title': title,
            'type': types[i],
            'price': prices[i].split(' ')[0],
            'solddate': solddate
        }
        type_list.append(type_info)

    return type_list

#(['Ticket Type', '2 DEC: GOODMOOD', '3 DEC: GOODMOOD', '4 DEC: GOODMOOD', '8 DEC: GOODMOOD', '9 DEC: GOODMOOD', '10 DEC: GOODMOOD', '11 DEC: GOODMOOD', '15 DEC: GOODMOOD', '16 DEC: GOODMOOD', '17 DEC: GOODMOOD', '[BUNDLE] 2 DEC: GOODMOOD & GOODFOOD', '[BUNDLE] 3 DEC: GOODMOOD & GOODFOOD', '[BUNDLE] 4 DEC: GOODMOOD & GOODFOOD', '[BUNDLE] 8 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 9 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 10 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 11 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 15 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 16 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 17 DEC: GOODMOOD & GOODHOOD'], 
# ['Price', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB'], 
# ['Qty.', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', ' 0 123456789101112131415', ' 0 123456789101112131415', ' 0 123456789101112131415', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', ' 0 123456789101112131415', ' 0 123456789101112131415', ' 0 123456789101112131415'])

In [327]:
unique_concert = df['title'].unique()
today_date = datetime.now()
types_list = []
count =0
for i in range(len(unique_concert)) :
    count +=1
    title = unique_concert[i]
    starttime = time.time()
    print(title)
    enddate = df.loc[df['title']==title, 'enddate'].iloc[0] + timedelta(days=1) #add 1 days to include the enddate
    if enddate < today_date: continue
    url = df.loc[df['title']==title, 'url'].iloc[0]
    soup = getSoup(url)
    types, prices, qtys = getInfo(soup)
    endtime = time.time()
    print(endtime - starttime)
    if len(types) == 0 :         
        print('count =',count)
        count =0
        time.sleep(660)
        print('-------------------------------------------')

        starttime = time.time()
        print(title)
        soup = getSoup(url)
        types, prices, qtys = getInfo(soup)
        type_list = createTypelist(title, types, prices, qtys)
        types_list += type_list
        endtime = time.time()
        print(endtime - starttime)
        print('-------------------------------------------')
        continue
    print('-------------------------------------------')
    type_list = createTypelist(title, types, prices, qtys)
    types_list += type_list

temp = pd.DataFrame(types_list)
df = pd.merge(df, temp, on='title', how='outer')

Monet and Friends Alive Bangkok
13.337769031524658
-------------------------------------------
Enchanted Flora Park
17.077724933624268
-------------------------------------------
The Backyard 168: Back to the 90's Campfire Party
9.169781684875488
count = 3
-------------------------------------------
The Backyard 168: Back to the 90's Campfire Party
19.481935024261475
-------------------------------------------
GOODMOOD
15.226334810256958
-------------------------------------------
GOODHOOD VOLUME 04
14.644158363342285
-------------------------------------------
Best Comedy Show On Khaosan - Davidson Boswell
11.597674369812012
-------------------------------------------
NuNew 2nd Single Nu Meow Official Goods
19.983825206756592
-------------------------------------------
คอนเสิร์ต อ๊อฟ ปองศักดิ์ Concert Aof Pongsak
11.795042037963867
-------------------------------------------
Rasmee Isan Soul LIVE at Thapae East
11.879853963851929
-------------------------------------------
The Melody 

In [338]:
# df = pd.merge(df, temp, on='title', how='outer')

In [340]:
pd.set_option('display.max_row', None)

In [350]:
date = datetime.now().date()
print(date)
path = '/Users/got/Documents/retune/t+im/melon_solddate/melonSolddate'+'('+str(date)+')'+'.csv'
# print(path)
df.to_csv(path, index=False)

2023-12-13


# check solddate

- get old df
- scrape melon page for maore concert
- check solddate